### Практика по groupby.

In [ ]:
df = pd.read_csv("legislators_historical.csv")
df.columns

In [ ]:
import pandas as pd

dtypes = {
    "first_name": "category",
    "last_name": "category",
    "gender_bio": "category",
    "religion_bio": "category",
    "suffix_name": "category",
}
df = pd.read_csv(
    "legislators_historical.csv",
    dtype=dtypes,
    usecols = ['last_name', 'first_name', 'gender_bio', 'birthday_bio', 'religion_bio', \
               'suffix_name']
)

df.head()

In [ ]:
df.dtypes

Для каждой религии (religion_bio) вычислите количество её приверженцев.

In [ ]:
n_by_religion = #your code here
n_by_religion.head(10)

Вычислите количество приверженцев каждой религии каждого пола.

In [ ]:
#your code here

После применения groupby получаем объект типа pandas Series.

In [ ]:
type(n_by_religion)

Преобразуйте объект n_by_religion в dataframe.

In [ ]:
n_by_religion_df = #your code here
n_by_religion_df.head(3)

Выведите на экран для каждой религии last_name и gender_bio первого человека из df, который исповедует эту религию. На выходе получите dataframe.

In [ ]:
df_gen_name = #your code here
df_gen_name.head(3)

К полученной таблице примените groupby, чтобы выяснить, сколько раз каждый пол встречается в этой таблице.

In [ ]:
#your code here

Заполните пропуски в столбце birthday_bio датой '1950-00-00'. 

Создайте столбец age, в котором вычислите возраст (в годах) каждого человека.

С помощью groupby вычислите средний возраст для приверженцев каждой религии.

In [ ]:
#your code here

***np.in1d(array1, array2) - для каждого элемента array1 проверяет, присутствует ли он в array2.***

In [ ]:
import numpy as np

test = np.array([0, 1, 2, 5, 0])
states = [0, 2]
mask = np.in1d(test, states)
mask

### Рекомендательные системы. Часть 1.

В этом задании будем практиковаться в реализации рекомендательных систем.

Воспользуемся небольшим датасетом с Kaggle: [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

## Часть 0. Загрузка данных

Загрузим [Deskdrop dataset](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop), включающийся в себе логи за 1 год платформы, где пользователи читают статьи.

Данные включают в себя 2 файла:  
- **shared_articles.csv**
- **users_interactions.csv**

Как можно догадаться, в одном описания самих статей (нам понадобятся в контентных моделях), а в другом логи пользователей.

#### shared_articles.csv

Так как в файле перечислены даже удалённые статьи, то мы их сразу удалим (на самом деле они могли бы быть нам полезны для подсчёта различных величин, хоть мы и не можем их рекомендовать).

In [ ]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(5)

#### users_interactions.csv

В колонке eventType описаны действия, которые могли совершать пользователи над статьёй:  
- VIEW
- LIKE
- COMMENT CREATED
- FOLLOW
- BOOKMARK

In [ ]:
interactions_df = pd.read_csv('users_interactions.csv')
interactions_df.head(10)

In [ ]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

### Предобработка данных

В логах встречаются различные действия пользователей. Однако мы хотим работать лишь с одной величиной, характеризующей всё взаимодействие пользователя со статьёй. Предлагается задать действиям следующие веса:

In [ ]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

Посчитайте числовую величину "оценки" пользователем статьи с указанными выше весами.

In [ ]:
interactions_df['eventStrength']  = #your code here

In [ ]:
interactions_df.eventStrength.hist(bins=20)

В данной постановке каждый пользователей мог взаимодействовать с каждой статьёй более 1 раза (как минимум совершая различные действия). Предлагается "схлопнуть" все действия в одно взаимодействие с весом, равным сумме весов. 

Однако полученное число будет в том числе тем больше, чем больше действий произвёл человек. Чтобы уменьшить разброс предлагается взять логарифм от полученного числа (можно придумыват другие веса действиям и по-другому обрабатывать значения).

Также сохраним последнее значение времени взаимодействия для разделениея выборки на обучение и контроль.

***С помощью groupby создайте объект interactions_full_df, в котором для каждой пары personId-contentId посчитайте сумму взаимодействий (eventStrength), затем примените к нему сглаживание (smooth_user_preference) и сохраните как dataframe со столбцами personId и contentId.***

In [ ]:
users_interactions_count_df = #your code here
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = #your code here
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

Оставьте только те взаимодействия из таблицы interactions_df, которые касаются только отфильтрованных пользователей.

In [ ]:
interactions_from_selected_users_df = #your code here

In [ ]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

In [ ]:
users_interactions_count_df.hist(bins=20)

В данной постановке каждый пользователей мог взаимодействовать с каждой статьёй более 1 раза (как минимум совершая различные действия). Предлагается "схлопнуть" все действия в одно взаимодействие с весом, равным сумме весов. 

Однако полученное число будет в том числе тем больше, чем больше действий произвёл человек. Чтобы уменьшить разброс предлагается взять логарифм от полученного числа (можно придумыват другие веса действиям и по-другому обрабатывать значения).

Также сохраним последнее значение времени взаимодействия для разделениея выборки на обучение и контроль.

***С помощью groupby создайте объект interactions_full_df, в котором для каждой пары personId-contentId посчитайте сумму взаимодействий (eventStrength), затем примените к нему сглаживание (smooth_user_preference) и сохраните как dataframe с колонками-индексами personId и contentId.***

In [ ]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = #your code here

interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(10)

Разбейте выборку на обучение и контроль по времени (train - объекты с last_timestamp < split_ts, test - остальные объекты).

In [ ]:
split_ts = 1475519530
interactions_train_df = #your code here
interactions_test_df = #your code here

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [ ]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(1)

## Часть 1: Baseline (модель по популярности)

Самой простой моделью рекомендаций (при этом достаточно сильной!) является модель, которая рекомендует наиболее популярные статьи. 

Предлагается реализовать её. Давайте считать, что рекомендуем мы по 10 материалов (такое ограничение на размер блока на сайте).

In [ ]:
interactions_train_df.head()

Посчитайте популярность каждой статьи, как сумму всех "оценок" взаимодействий с ней. Отсортируйте материалы по убыванию их популярности. Как результат создайте массив popular_content, состоящий из отсортированных по популярности статей.

In [ ]:
popular_content = #your code here

In [ ]:
popular_content

Теперь необходимо сделать предсказания для каждого пользователя. Не забывайте, что надо рекомендовать то, что пользователь ещё не читал (для этого нужно проверить, что материал не встречался в true_train).

In [ ]:
top_k = 10

interactions['prediction_popular'] = (
    interactions.true_train
    .apply(
        lambda x:
        popular_content[~np.in1d(popular_content, x)][:top_k]
    )
)

Настало время оценить качество. Посчитаем precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредните по всем пользователям. Везде далее будем считать эту же метрику.

(для каждого пользователя посчитаем долю правильно предсказанных статей (если в ответе их меньше 10ти, то будем нормировать на реальное количество статей)).

In [ ]:
def calc_precision(column):
    return (
        interactions
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

In [ ]:
calc_precision('prediction_popular')

In [ ]:
interactions.true_test.apply(len).sum()

## Часть 2. Коллаборативная фильтрация.

Перейдём к более сложному механизму рекомендаций, а именно коллаборативной фильтрации. Суть коллаборативной фильтрации в том, что учитывается схожесть пользователей и товаров между собой, а не факторы, которые их описывают. 

Реализуем memory-based подход.

Для начала для удобства составим матрицу "оценок" пользователей. Нули будут обозначать отсутствие взаимодействия.

In [ ]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)

In [ ]:
ratings.head()

### Memory-based

Посчитайте схожести пользователей с помощью корреляции Пирсона. Для каждой пары учитываем только ненулевые значения.

Для скорости работы лучше переходить от pandas к numpy.

In [ ]:
ratings_m = ratings.as_matrix()

Дозаполните код: 

1) Отнормируйте рейтинги (поделите рейтинг на его норму).

2) Вычислите корреляцию между пользователями.

In [ ]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
        
        # nonzero elements of two users
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        
        # continue if no intersection
        if np.sum(mask_uv) == 0:
            continue
            
        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]
        
        # normalization
        # YOUR CODE HERE
        
        # for nonzero std
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        
        similarity_users[i,j] = # YOUR CODE HERE
        similarity_users[j,i] = # YOUR CODE HERE

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для рекомендаций.

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждой статьи долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

После того, как будут сделаны предсказания (новый столбец в interactions), посчитайте качество по той же метрике.

In [1]:
#1. users_sim - массив, где похожие на данного пользователя пользователи 
#   отмечены как True, а остальные как False.

#2. tmp_recommend - статьи, с которыми взаимодействовали пользователи, похожие на нашего.
#   для подсчета tmp_recommend:
#   1. Оставьте в матрице ratings_m только похожих пользователей (users_sim)
#   2. Просуммируйте значения в строках. Затем отсортируйте номера строк (np.argsort) 
#      по убыванию (популярности).
#   3. Оставьте только колонки tmp_recommend в матрице ratings (наиболее популярные статьи)
#      и сохраните их как np.array под названием tmp_recommend_arr.

#3. recommend - 10 статей с наибольшими долями. Исключите из tmp_recommend_arr статьи, 
#   которые пользователь уже видел.

In [ ]:
prediction_user_based = []
for i in tqdm_notebook(range(len(similarity_users))):
        
    users_sim = #your code here

    if sum(users_sim) == 0:
        prediction_user_based.append([])
    else:
        tmp_recommend = #your code here
        tmp_recommend_arr = #your code here
        recommend = #your code here
        prediction_user_based.append(list(recommend))

interactions['prediction_user_based'] = prediction_user_based

In [ ]:
calc_precision('prediction_user_based')